This is to help work through some plex issues.

In [12]:
#!pwsh

$cluster = "admin@equestria"
$namespace = "equestria"
$pvcName = "plex"
$podName = "plex-debug"

kubectl config use-context $cluster

Switched to context "admin@equestria".


In [13]:
#!pwsh

$debugPodYaml = @"
apiVersion: v1
kind: Pod
metadata:
  name: $podName
  namespace: $namespace
  labels:
    app: plex-debug
spec:
  restartPolicy: Never
  securityContext:
    runAsUser: 568
    runAsGroup: 568
    fsGroup: 568
    fsGroupChangePolicy: OnRootMismatch
    supplementalGroups: [44, 109, 303, 568, 10000]
  containers:
  - name: toolbox
    image: 1.25.1-bookworm@sha256:c4bc0741e3c79c0e2d47ca2505a06f5f2a44682ada94e1dba251a3854e60c2bd
    command: ["/bin/bash"]
    args:
      - -c
      - |
        echo "🚀 Debug pod ready. PVC mounted at /config"
        echo "📁 Contents:"
        ls -lah /config
        echo ""
        echo "💤 Keeping pod alive. Use 'kubectl exec' to access."
        trap : TERM INT; sleep infinity & wait
    securityContext:
      allowPrivilegeEscalation: true
    volumeMounts:
    - name: plex-data
      mountPath: /config
  volumes:
  - name: plex-data
    persistentVolumeClaim:
      claimName: $pvcName
"@

Write-Host "🚀 Creating debug pod..." -ForegroundColor Cyan
$debugPodYaml | kubectl apply -f -

Write-Host "`n⏳ Waiting for pod to be ready..."
kubectl wait --for=condition=Ready pod/$podName -n $namespace --timeout=120s

Write-Host "`n✅ Debug pod ready!" -ForegroundColor Green
kubectl get pod $podName -n $namespace

Write-Host "🔧 Opening shell in debug pod..." -ForegroundColor Cyan
Write-Host "Plex config is at: /config" -ForegroundColor Yellow

🚀 Creating debug pod...

⏳ Waiting for pod to be ready...

✅ Debug pod ready!
🔧 Opening shell in debug pod...
Plex config is at: /config
pod/plex-debug created
pod/plex-debug condition met
NAME         READY   STATUS    RESTARTS   AGE
plex-debug   1/1     Running   0          2s


In [24]:
kubectl exec -it $podName -n $namespace -- ls -al "/config"


Unable to use a TTY - input is not a terminal or the right kind of file Exception System.Management.Automation.RemoteException: Unable to use a TTY - input is not a terminal or the right kind of file SerializedRemoteException <null> SerializedRemoteInvocationInfo <null> ErrorRecord Unable to use a TTY - input is not a terminal or the right kind of file Exception System.Management.Automation.ParentContainsErrorRecordException: Unable to use a TTY - input is not a terminal or the right kind of file Message Unable to use a TTY - input is not a terminal or the right kind of file TargetSite <null> Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotSpecified: (:) [], ParentContainsErrorRecordException Category NotSpecified Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId RuntimeException ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] WasThrownFromThrowStatement False TargetSite <null> Message Unable to use a TTY - input is not a terminal or the right kind of file Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject Unable to use a TTY - input is not a terminal or the right kind of file CategoryInfo NotSpecified: (Unable to use a TTY… right kind of file:String) [], RemoteException Category NotSpecified Activity Reason RemoteException TargetName Unable to use a TTY - input is not a terminal or the right kind of file TargetType String FullyQualifiedErrorId NativeCommandError ErrorDetails <null> InvocationInfo System.Management.Automation.InvocationInfo MyCommand kubectl Path /opt/homebrew/bin/kubectl Extension Definition /opt/homebrew/bin/kubectl Source /opt/homebrew/bin/kubectl Version 0.0.0.0 Major 0 Minor 0 Build 0 Revision 0 MajorRevision 0 MinorRevision 0 Visibility Public OutputType index value 0 System.String Name System.String Type System.String TypeDefinitionAst <null> Name kubectl CommandType Application ModuleName Module <null> RemotingCapability PowerShell Parameters System.Management.Automation.PSNotSupportedException: Cannot perform operation because operation "NewNotSupportedException at offset 88 in file:line:column <filename unknown>:0:0\n" is not supported.\n at System.Management.Automation.CommandMetadata..ctor(CommandInfo commandInfo, Boolean shouldGen... ErrorRecord Cannot perform operation because operation "NewNotSupportedException at offset 88 in file:line:column <filename unknown>:0:0\n" is not supported. Exception System.Management.Automation.ParentContainsErrorRecordException: Cannot perform operation because operation "NewNotSupportedException at offset 88 in file:line:column <filename unknown>:0:0\n" is not supported. Message Cannot perform operation because operation "NewNotSupportedException at offset 88 in file:line:column :0:0
" is not supported. TargetSite <null> Data [ ] InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotImplemented: (:) [], ParentContainsErrorRecordException Category NotImplemented Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId NotSupported ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] TargetSite Void .ctor(System.Management.Automation.CommandInfo, Boolean) Name .ctor MemberType Constructor DeclaringType System.Management.Automation.CommandMetadata ReflectedType System.Management.Automation.CommandMetadata MetadataToken 100667362 Module System.Management.Automation.dll MDStreamVersion 131072 FullyQualifiedName /Users/david/.nuget/packages/microsoft.dotnet-interactive/1.0.632301/tools/net9.0/any/runtimes/unix/lib/net9.0/System.Management.Automation.dll ModuleVersionId 6124f3ae-295f-4d47-a897-fbada8c885b7 MetadataToken 1 ScopeName System.Management.Automation.dll Name System.Managem

total 16
drwxrwxrwx. 2 root root 4096 Oct 14 02:53 .
drwxr-xr-x. 1 root root   42 Oct 14 02:40 ..
-rw-------. 1  568  568   42 Oct 14 02:00 .LocalAdminToken
-rw-------. 1  568  568 1779 Oct 14 02:32 Preferences.xml
-rw-r--r--. 1  568  568    1 Oct 14 02:00 plexmediaserver.pid


In [23]:
kubectl exec -it $podName -n $namespace -- rm -rf "/config/Crash Reports"
kubectl exec -it $podName -n $namespace -- rm -rf "/config/Plug-in Support"
kubectl exec -it $podName -n $namespace -- rm -rf "/config/Setup Plex.html"

Unable to use a TTY - input is not a terminal or the right kind of file Exception System.Management.Automation.RemoteException: Unable to use a TTY - input is not a terminal or the right kind of file SerializedRemoteException <null> SerializedRemoteInvocationInfo <null> ErrorRecord Unable to use a TTY - input is not a terminal or the right kind of file Exception System.Management.Automation.ParentContainsErrorRecordException: Unable to use a TTY - input is not a terminal or the right kind of file Message Unable to use a TTY - input is not a terminal or the right kind of file TargetSite <null> Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotSpecified: (:) [], ParentContainsErrorRecordException Category NotSpecified Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId RuntimeException ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] WasThrownFromThrowStatement False TargetSite <null> Message Unable to use a TTY - input is not a terminal or the right kind of file Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject Unable to use a TTY - input is not a terminal or the right kind of file CategoryInfo NotSpecified: (Unable to use a TTY… right kind of file:String) [], RemoteException Category NotSpecified Activity Reason RemoteException TargetName Unable to use a TTY - input is not a terminal or the right kind of file TargetType String FullyQualifiedErrorId NativeCommandError ErrorDetails <null> InvocationInfo System.Management.Automation.InvocationInfo MyCommand kubectl Path /opt/homebrew/bin/kubectl Extension Definition /opt/homebrew/bin/kubectl Source /opt/homebrew/bin/kubectl Version 0.0.0.0 Major 0 Minor 0 Build 0 Revision 0 MajorRevision 0 MinorRevision 0 Visibility Public OutputType index value 0 System.String Name System.String Type System.String TypeDefinitionAst <null> Name kubectl CommandType Application ModuleName Module <null> RemotingCapability PowerShell Parameters System.Management.Automation.PSNotSupportedException: Cannot perform operation because operation "NewNotSupportedException at offset 88 in file:line:column <filename unknown>:0:0\n" is not supported.\n at System.Management.Automation.CommandMetadata..ctor(CommandInfo commandInfo, Boolean shouldGen... ErrorRecord Cannot perform operation because operation "NewNotSupportedException at offset 88 in file:line:column <filename unknown>:0:0\n" is not supported. Exception System.Management.Automation.ParentContainsErrorRecordException: Cannot perform operation because operation "NewNotSupportedException at offset 88 in file:line:column <filename unknown>:0:0\n" is not supported. Message Cannot perform operation because operation "NewNotSupportedException at offset 88 in file:line:column :0:0
" is not supported. TargetSite <null> Data [ ] InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotImplemented: (:) [], ParentContainsErrorRecordException Category NotImplemented Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId NotSupported ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] TargetSite Void .ctor(System.Management.Automation.CommandInfo, Boolean) Name .ctor MemberType Constructor DeclaringType System.Management.Automation.CommandMetadata ReflectedType System.Management.Automation.CommandMetadata MetadataToken 100667362 Module System.Management.Automation.dll MDStreamVersion 131072 FullyQualifiedName /Users/david/.nuget/packages/microsoft.dotnet-interactive/1.0.632301/tools/net9.0/any/runtimes/unix/lib/net9.0/System.Management.Automation.dll ModuleVersionId 6124f3ae-295f-4d47-a897-fbada8c885b7 MetadataToken 1 ScopeName System.Management.Automation.dll Name System.Managem

Unable to use a TTY - input is not a terminal or the right kind of file Exception System.Management.Automation.RemoteException: Unable to use a TTY - input is not a terminal or the right kind of file SerializedRemoteException <null> SerializedRemoteInvocationInfo <null> ErrorRecord Unable to use a TTY - input is not a terminal or the right kind of file Exception System.Management.Automation.ParentContainsErrorRecordException: Unable to use a TTY - input is not a terminal or the right kind of file Message Unable to use a TTY - input is not a terminal or the right kind of file TargetSite <null> Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotSpecified: (:) [], ParentContainsErrorRecordException Category NotSpecified Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId RuntimeException ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] WasThrownFromThrowStatement False TargetSite <null> Message Unable to use a TTY - input is not a terminal or the right kind of file Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject Unable to use a TTY - input is not a terminal or the right kind of file CategoryInfo NotSpecified: (Unable to use a TTY… right kind of file:String) [], RemoteException Category NotSpecified Activity Reason RemoteException TargetName Unable to use a TTY - input is not a terminal or the right kind of file TargetType String FullyQualifiedErrorId NativeCommandError ErrorDetails <null> InvocationInfo System.Management.Automation.InvocationInfo MyCommand kubectl Path /opt/homebrew/bin/kubectl Extension Definition /opt/homebrew/bin/kubectl Source /opt/homebrew/bin/kubectl Version 0.0.0.0 Major 0 Minor 0 Build 0 Revision 0 MajorRevision 0 MinorRevision 0 Visibility Public OutputType index value 0 System.String Name System.String Type System.String TypeDefinitionAst <null> Name kubectl CommandType Application ModuleName Module <null> RemotingCapability PowerShell Parameters System.Management.Automation.PSNotSupportedException: Cannot perform operation because operation "NewNotSupportedException at offset 88 in file:line:column <filename unknown>:0:0\n" is not supported.\n at System.Management.Automation.CommandMetadata..ctor(CommandInfo commandInfo, Boolean shouldGen... ErrorRecord Cannot perform operation because operation "NewNotSupportedException at offset 88 in file:line:column <filename unknown>:0:0\n" is not supported. Exception System.Management.Automation.ParentContainsErrorRecordException: Cannot perform operation because operation "NewNotSupportedException at offset 88 in file:line:column <filename unknown>:0:0\n" is not supported. Message Cannot perform operation because operation "NewNotSupportedException at offset 88 in file:line:column :0:0
" is not supported. TargetSite <null> Data [ ] InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotImplemented: (:) [], ParentContainsErrorRecordException Category NotImplemented Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId NotSupported ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] TargetSite Void .ctor(System.Management.Automation.CommandInfo, Boolean) Name .ctor MemberType Constructor DeclaringType System.Management.Automation.CommandMetadata ReflectedType System.Management.Automation.CommandMetadata MetadataToken 100667362 Module System.Management.Automation.dll MDStreamVersion 131072 FullyQualifiedName /Users/david/.nuget/packages/microsoft.dotnet-interactive/1.0.632301/tools/net9.0/any/runtimes/unix/lib/net9.0/System.Management.Automation.dll ModuleVersionId 6124f3ae-295f-4d47-a897-fbada8c885b7 MetadataToken 1 ScopeName System.Management.Automation.dll Name System.Managem

Unable to use a TTY - input is not a terminal or the right kind of file Exception System.Management.Automation.RemoteException: Unable to use a TTY - input is not a terminal or the right kind of file SerializedRemoteException <null> SerializedRemoteInvocationInfo <null> ErrorRecord Unable to use a TTY - input is not a terminal or the right kind of file Exception System.Management.Automation.ParentContainsErrorRecordException: Unable to use a TTY - input is not a terminal or the right kind of file Message Unable to use a TTY - input is not a terminal or the right kind of file TargetSite <null> Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotSpecified: (:) [], ParentContainsErrorRecordException Category NotSpecified Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId RuntimeException ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] WasThrownFromThrowStatement False TargetSite <null> Message Unable to use a TTY - input is not a terminal or the right kind of file Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject Unable to use a TTY - input is not a terminal or the right kind of file CategoryInfo NotSpecified: (Unable to use a TTY… right kind of file:String) [], RemoteException Category NotSpecified Activity Reason RemoteException TargetName Unable to use a TTY - input is not a terminal or the right kind of file TargetType String FullyQualifiedErrorId NativeCommandError ErrorDetails <null> InvocationInfo System.Management.Automation.InvocationInfo MyCommand kubectl Path /opt/homebrew/bin/kubectl Extension Definition /opt/homebrew/bin/kubectl Source /opt/homebrew/bin/kubectl Version 0.0.0.0 Major 0 Minor 0 Build 0 Revision 0 MajorRevision 0 MinorRevision 0 Visibility Public OutputType index value 0 System.String Name System.String Type System.String TypeDefinitionAst <null> Name kubectl CommandType Application ModuleName Module <null> RemotingCapability PowerShell Parameters System.Management.Automation.PSNotSupportedException: Cannot perform operation because operation "NewNotSupportedException at offset 88 in file:line:column <filename unknown>:0:0\n" is not supported.\n at System.Management.Automation.CommandMetadata..ctor(CommandInfo commandInfo, Boolean shouldGen... ErrorRecord Cannot perform operation because operation "NewNotSupportedException at offset 88 in file:line:column <filename unknown>:0:0\n" is not supported. Exception System.Management.Automation.ParentContainsErrorRecordException: Cannot perform operation because operation "NewNotSupportedException at offset 88 in file:line:column <filename unknown>:0:0\n" is not supported. Message Cannot perform operation because operation "NewNotSupportedException at offset 88 in file:line:column :0:0
" is not supported. TargetSite <null> Data [ ] InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotImplemented: (:) [], ParentContainsErrorRecordException Category NotImplemented Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId NotSupported ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] TargetSite Void .ctor(System.Management.Automation.CommandInfo, Boolean) Name .ctor MemberType Constructor DeclaringType System.Management.Automation.CommandMetadata ReflectedType System.Management.Automation.CommandMetadata MetadataToken 100667362 Module System.Management.Automation.dll MDStreamVersion 131072 FullyQualifiedName /Users/david/.nuget/packages/microsoft.dotnet-interactive/1.0.632301/tools/net9.0/any/runtimes/unix/lib/net9.0/System.Management.Automation.dll ModuleVersionId 6124f3ae-295f-4d47-a897-fbada8c885b7 MetadataToken 1 ScopeName System.Management.Automation.dll Name System.Managem

In [17]:
#!pwsh

Write-Host "⚠️  Scaling down Plex..." -ForegroundColor Yellow
kubectl scale deployment plex -n $namespace --replicas=0

Write-Host "`nWaiting for Plex pod to terminate..."
kubectl wait --for=delete pod -n $namespace -l app.kubernetes.io/name=plex --timeout=120s

Write-Host "✅ Plex scaled down" -ForegroundColor Green


⚠️  Scaling down Plex...

Waiting for Plex pod to terminate...
✅ Plex scaled down
deployment.apps/plex scaled
pod/plex-ccf674f6c-lln9k condition met


In [7]:
#!pwsh

# $backupDate = "2025-09-30"

# kubectl exec -it $podName -n $namespace -- cp "/config/Plug-in Support/Databases/com.plexapp.plugins.library.db-$($backupDate)" "/config/Plug-in Support/Databases/com.plexapp.plugins.library.db"
# kubectl exec -it $podName -n $namespace -- cp "/config/Plug-in Support/Databases/com.plexapp.plugins.library.blobs.db-$($backupDate)" "/config/Plug-in Support/Databases/com.plexapp.plugins.library.blobs.db"
kubectl exec -it $podName -n $namespace -- rm "/config/Plug-in Support/Databases/com.plexapp.plugins.library.blobs.db"
kubectl exec -it $podName -n $namespace -- rm "/config/Plug-in Support/Databases/com.plexapp.plugins.library.blobs.db-shm"
kubectl exec -it $podName -n $namespace -- rm "/config/Plug-in Support/Databases/com.plexapp.plugins.library.blobs.db-wal"
kubectl exec -it $podName -n $namespace -- rm "/config/Plug-in Support/Databases/com.plexapp.plugins.library.db"
kubectl exec -it $podName -n $namespace -- rm "/config/Plug-in Support/Databases/com.plexapp.plugins.library.db-shm"
kubectl exec -it $podName -n $namespace -- rm "/config/Plug-in Support/Databases/com.plexapp.plugins.library.db-wal"


Unable to use a TTY - input is not a terminal or the right kind of file Exception System.Management.Automation.RemoteException: Unable to use a TTY - input is not a terminal or the right kind of file SerializedRemoteException <null> SerializedRemoteInvocationInfo <null> ErrorRecord Unable to use a TTY - input is not a terminal or the right kind of file Exception System.Management.Automation.ParentContainsErrorRecordException: Unable to use a TTY - input is not a terminal or the right kind of file Message Unable to use a TTY - input is not a terminal or the right kind of file TargetSite <null> Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotSpecified: (:) [], ParentContainsErrorRecordException Category NotSpecified Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId RuntimeException ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] WasThrownFromThrowStatement False TargetSite <null> Message Unable to use a TTY - input is not a terminal or the right kind of file Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject Unable to use a TTY - input is not a terminal or the right kind of file CategoryInfo NotSpecified: (Unable to use a TTY… right kind of file:String) [], RemoteException Category NotSpecified Activity Reason RemoteException TargetName Unable to use a TTY - input is not a terminal or the right kind of file TargetType String FullyQualifiedErrorId NativeCommandError ErrorDetails <null> InvocationInfo System.Management.Automation.InvocationInfo MyCommand kubectl Path /opt/homebrew/bin/kubectl Extension Definition /opt/homebrew/bin/kubectl Source /opt/homebrew/bin/kubectl Version 0.0.0.0 Major 0 Minor 0 Build 0 Revision 0 MajorRevision 0 MinorRevision 0 Visibility Public OutputType index value 0 System.String Name System.String Type System.String TypeDefinitionAst <null> Name kubectl CommandType Application ModuleName Module <null> RemotingCapability PowerShell Parameters System.Management.Automation.PSNotSupportedException: Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column <filename unknown>:0:0\n" is not supported.\n at System.Management.Automation.CommandMetadata..ctor(CommandInfo commandInfo, Boolean shouldGen... ErrorRecord Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column <filename unknown>:0:0\n" is not supported. Exception System.Management.Automation.ParentContainsErrorRecordException: Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column <filename unknown>:0:0\n" is not supported. Message Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column :0:0
" is not supported. TargetSite <null> Data [ ] InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotImplemented: (:) [], ParentContainsErrorRecordException Category NotImplemented Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId NotSupported ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] TargetSite Void .ctor(System.Management.Automation.CommandInfo, Boolean) Name .ctor MemberType Constructor DeclaringType System.Management.Automation.CommandMetadata ReflectedType System.Management.Automation.CommandMetadata MetadataToken 100667362 Module System.Management.Automation.dll MDStreamVersion 131072 FullyQualifiedName /Users/david/.nuget/packages/microsoft.dotnet-interactive/1.0.632301/tools/net9.0/any/runtimes/unix/lib/net9.0/System.Management.Automation.dll ModuleVersionId 6124f3ae-295f-4d47-a897-fbada8c885b7 MetadataToken 1 ScopeName System.Management.Automation.dll Name System.Managem

rm: cannot remove '/config/Plug-in Support/Databases/com.plexapp.plugins.library.blobs.db': No such file or directory Exception System.Management.Automation.RemoteException: rm: cannot remove '/config/Plug-in Support/Databases/com.plexapp.plugins.library.blobs.db': No such file or directory SerializedRemoteException <null> SerializedRemoteInvocationInfo <null> ErrorRecord rm: cannot remove '/config/Plug-in Support/Databases/com.plexapp.plugins.library.blobs.db': No such file or directory Exception System.Management.Automation.ParentContainsErrorRecordException: rm: cannot remove '/config/Plug-in Support/Databases/com.plexapp.plugins.library.blobs.db': No such file or directory Message rm: cannot remove '/config/Plug-in Support/Databases/com.plexapp.plugins.library.blobs.db': No such file or directory TargetSite <null> Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotSpecified: (:) [], ParentContainsErrorRecordException Category NotSpecified Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId RuntimeException ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] WasThrownFromThrowStatement False TargetSite <null> Message rm: cannot remove '/config/Plug-in Support/Databases/com.plexapp.plugins.library.blobs.db': No such file or directory Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotSpecified: (:) [], RemoteException Category NotSpecified Activity Reason RemoteException TargetName TargetType FullyQualifiedErrorId NativeCommandErrorMessage ErrorDetails <null> InvocationInfo System.Management.Automation.InvocationInfo MyCommand kubectl Path /opt/homebrew/bin/kubectl Extension Definition /opt/homebrew/bin/kubectl Source /opt/homebrew/bin/kubectl Version 0.0.0.0 Major 0 Minor 0 Build 0 Revision 0 MajorRevision 0 MinorRevision 0 Visibility Public OutputType index value 0 System.String Name System.String Type System.String TypeDefinitionAst <null> Name kubectl CommandType Application ModuleName Module <null> RemotingCapability PowerShell Parameters System.Management.Automation.PSNotSupportedException: Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column <filename unknown>:0:0\n" is not supported.\n at System.Management.Automation.CommandMetadata..ctor(CommandInfo commandInfo, Boolean shouldGen... ErrorRecord Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column <filename unknown>:0:0\n" is not supported. Exception System.Management.Automation.ParentContainsErrorRecordException: Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column <filename unknown>:0:0\n" is not supported. Message Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column :0:0
" is not supported. TargetSite <null> Data [ ] InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotImplemented: (:) [], ParentContainsErrorRecordException Category NotImplemented Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId NotSupported ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] TargetSite Void .ctor(System.Management.Automation.CommandInfo, Boolean) Name .ctor MemberType Constructor DeclaringType System.Management.Automation.CommandMetadata ReflectedType System.Management.Automation.CommandMetadata MetadataToken 100667362 Module System.Management.Automation.dll MDStreamVersion 131072 FullyQualifiedName /Users/david/.nuget/packages/microsoft.dotnet-interactive/1.0.632301/tools/net9.0/any/runtimes/unix/lib/net9.0/System.Management.Automation.dll ModuleVersionId 6124f3ae-295f-4d47-a8

command terminated with exit code 1 Exception System.Management.Automation.RemoteException: command terminated with exit code 1 SerializedRemoteException <null> SerializedRemoteInvocationInfo <null> ErrorRecord command terminated with exit code 1 Exception System.Management.Automation.ParentContainsErrorRecordException: command terminated with exit code 1 Message command terminated with exit code 1 TargetSite <null> Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotSpecified: (:) [], ParentContainsErrorRecordException Category NotSpecified Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId RuntimeException ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] WasThrownFromThrowStatement False TargetSite <null> Message command terminated with exit code 1 Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotSpecified: (:) [], RemoteException Category NotSpecified Activity Reason RemoteException TargetName TargetType FullyQualifiedErrorId NativeCommandErrorMessage ErrorDetails <null> InvocationInfo System.Management.Automation.InvocationInfo MyCommand kubectl Path /opt/homebrew/bin/kubectl Extension Definition /opt/homebrew/bin/kubectl Source /opt/homebrew/bin/kubectl Version 0.0.0.0 Major 0 Minor 0 Build 0 Revision 0 MajorRevision 0 MinorRevision 0 Visibility Public OutputType index value 0 System.String Name System.String Type System.String TypeDefinitionAst <null> Name kubectl CommandType Application ModuleName Module <null> RemotingCapability PowerShell Parameters System.Management.Automation.PSNotSupportedException: Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column <filename unknown>:0:0\n" is not supported.\n at System.Management.Automation.CommandMetadata..ctor(CommandInfo commandInfo, Boolean shouldGen... ErrorRecord Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column <filename unknown>:0:0\n" is not supported. Exception System.Management.Automation.ParentContainsErrorRecordException: Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column <filename unknown>:0:0\n" is not supported. Message Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column :0:0
" is not supported. TargetSite <null> Data [ ] InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotImplemented: (:) [], ParentContainsErrorRecordException Category NotImplemented Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId NotSupported ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] TargetSite Void .ctor(System.Management.Automation.CommandInfo, Boolean) Name .ctor MemberType Constructor DeclaringType System.Management.Automation.CommandMetadata ReflectedType System.Management.Automation.CommandMetadata MetadataToken 100667362 Module System.Management.Automation.dll MDStreamVersion 131072 FullyQualifiedName /Users/david/.nuget/packages/microsoft.dotnet-interactive/1.0.632301/tools/net9.0/any/runtimes/unix/lib/net9.0/System.Management.Automation.dll ModuleVersionId 6124f3ae-295f-4d47-a897-fbada8c885b7 MetadataToken 1 ScopeName System.Management.Automation.dll Name System.Management.Automation.dll Assembly System.Management.Automation, Version=7.5.0.500, Culture=neutral, PublicKeyToken=31bf3856ad364e35 ModuleHandle System.ModuleHandle CustomAttributes [ [System.Security.UnverifiableCodeAttribute()], [System.Runtime.CompilerServices.RefSafetyRulesAttribute((Int32)11)] ] MethodHandle System.RuntimeMethodHandle Value 4536883264 Attributes Public, HideBySig, SpecialName, RT

Unable to use a TTY - input is not a terminal or the right kind of file Exception System.Management.Automation.RemoteException: Unable to use a TTY - input is not a terminal or the right kind of file SerializedRemoteException <null> SerializedRemoteInvocationInfo <null> ErrorRecord Unable to use a TTY - input is not a terminal or the right kind of file Exception System.Management.Automation.ParentContainsErrorRecordException: Unable to use a TTY - input is not a terminal or the right kind of file Message Unable to use a TTY - input is not a terminal or the right kind of file TargetSite <null> Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotSpecified: (:) [], ParentContainsErrorRecordException Category NotSpecified Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId RuntimeException ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] WasThrownFromThrowStatement False TargetSite <null> Message Unable to use a TTY - input is not a terminal or the right kind of file Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject Unable to use a TTY - input is not a terminal or the right kind of file CategoryInfo NotSpecified: (Unable to use a TTY… right kind of file:String) [], RemoteException Category NotSpecified Activity Reason RemoteException TargetName Unable to use a TTY - input is not a terminal or the right kind of file TargetType String FullyQualifiedErrorId NativeCommandError ErrorDetails <null> InvocationInfo System.Management.Automation.InvocationInfo MyCommand kubectl Path /opt/homebrew/bin/kubectl Extension Definition /opt/homebrew/bin/kubectl Source /opt/homebrew/bin/kubectl Version 0.0.0.0 Major 0 Minor 0 Build 0 Revision 0 MajorRevision 0 MinorRevision 0 Visibility Public OutputType index value 0 System.String Name System.String Type System.String TypeDefinitionAst <null> Name kubectl CommandType Application ModuleName Module <null> RemotingCapability PowerShell Parameters System.Management.Automation.PSNotSupportedException: Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column <filename unknown>:0:0\n" is not supported.\n at System.Management.Automation.CommandMetadata..ctor(CommandInfo commandInfo, Boolean shouldGen... ErrorRecord Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column <filename unknown>:0:0\n" is not supported. Exception System.Management.Automation.ParentContainsErrorRecordException: Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column <filename unknown>:0:0\n" is not supported. Message Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column :0:0
" is not supported. TargetSite <null> Data [ ] InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotImplemented: (:) [], ParentContainsErrorRecordException Category NotImplemented Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId NotSupported ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] TargetSite Void .ctor(System.Management.Automation.CommandInfo, Boolean) Name .ctor MemberType Constructor DeclaringType System.Management.Automation.CommandMetadata ReflectedType System.Management.Automation.CommandMetadata MetadataToken 100667362 Module System.Management.Automation.dll MDStreamVersion 131072 FullyQualifiedName /Users/david/.nuget/packages/microsoft.dotnet-interactive/1.0.632301/tools/net9.0/any/runtimes/unix/lib/net9.0/System.Management.Automation.dll ModuleVersionId 6124f3ae-295f-4d47-a897-fbada8c885b7 MetadataToken 1 ScopeName System.Management.Automation.dll Name System.Managem

rm: cannot remove '/config/Plug-in Support/Databases/com.plexapp.plugins.library.blobs.db-shm': No such file or directory Exception System.Management.Automation.RemoteException: rm: cannot remove '/config/Plug-in Support/Databases/com.plexapp.plugins.library.blobs.db-shm': No such file or directory SerializedRemoteException <null> SerializedRemoteInvocationInfo <null> ErrorRecord rm: cannot remove '/config/Plug-in Support/Databases/com.plexapp.plugins.library.blobs.db-shm': No such file or directory Exception System.Management.Automation.ParentContainsErrorRecordException: rm: cannot remove '/config/Plug-in Support/Databases/com.plexapp.plugins.library.blobs.db-shm': No such file or directory Message rm: cannot remove '/config/Plug-in Support/Databases/com.plexapp.plugins.library.blobs.db-shm': No such file or directory TargetSite <null> Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotSpecified: (:) [], ParentContainsErrorRecordException Category NotSpecified Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId RuntimeException ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] WasThrownFromThrowStatement False TargetSite <null> Message rm: cannot remove '/config/Plug-in Support/Databases/com.plexapp.plugins.library.blobs.db-shm': No such file or directory Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotSpecified: (:) [], RemoteException Category NotSpecified Activity Reason RemoteException TargetName TargetType FullyQualifiedErrorId NativeCommandErrorMessage ErrorDetails <null> InvocationInfo System.Management.Automation.InvocationInfo MyCommand kubectl Path /opt/homebrew/bin/kubectl Extension Definition /opt/homebrew/bin/kubectl Source /opt/homebrew/bin/kubectl Version 0.0.0.0 Major 0 Minor 0 Build 0 Revision 0 MajorRevision 0 MinorRevision 0 Visibility Public OutputType index value 0 System.String Name System.String Type System.String TypeDefinitionAst <null> Name kubectl CommandType Application ModuleName Module <null> RemotingCapability PowerShell Parameters System.Management.Automation.PSNotSupportedException: Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column <filename unknown>:0:0\n" is not supported.\n at System.Management.Automation.CommandMetadata..ctor(CommandInfo commandInfo, Boolean shouldGen... ErrorRecord Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column <filename unknown>:0:0\n" is not supported. Exception System.Management.Automation.ParentContainsErrorRecordException: Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column <filename unknown>:0:0\n" is not supported. Message Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column :0:0
" is not supported. TargetSite <null> Data [ ] InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotImplemented: (:) [], ParentContainsErrorRecordException Category NotImplemented Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId NotSupported ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] TargetSite Void .ctor(System.Management.Automation.CommandInfo, Boolean) Name .ctor MemberType Constructor DeclaringType System.Management.Automation.CommandMetadata ReflectedType System.Management.Automation.CommandMetadata MetadataToken 100667362 Module System.Management.Automation.dll MDStreamVersion 131072 FullyQualifiedName /Users/david/.nuget/packages/microsoft.dotnet-interactive/1.0.632301/tools/net9.0/any/runtimes/unix/lib/net9.0/System.Management.Automation.dll ModuleVersion

...

rm: cannot remove '/config/Plug-in Support/Databases/com.plexapp.plugins.library.db-shm': No such file or directory Exception System.Management.Automation.RemoteException: rm: cannot remove '/config/Plug-in Support/Databases/com.plexapp.plugins.library.db-shm': No such file or directory SerializedRemoteException <null> SerializedRemoteInvocationInfo <null> ErrorRecord rm: cannot remove '/config/Plug-in Support/Databases/com.plexapp.plugins.library.db-shm': No such file or directory Exception System.Management.Automation.ParentContainsErrorRecordException: rm: cannot remove '/config/Plug-in Support/Databases/com.plexapp.plugins.library.db-shm': No such file or directory Message rm: cannot remove '/config/Plug-in Support/Databases/com.plexapp.plugins.library.db-shm': No such file or directory TargetSite <null> Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotSpecified: (:) [], ParentContainsErrorRecordException Category NotSpecified Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId RuntimeException ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] WasThrownFromThrowStatement False TargetSite <null> Message rm: cannot remove '/config/Plug-in Support/Databases/com.plexapp.plugins.library.db-shm': No such file or directory Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotSpecified: (:) [], RemoteException Category NotSpecified Activity Reason RemoteException TargetName TargetType FullyQualifiedErrorId NativeCommandErrorMessage ErrorDetails <null> InvocationInfo System.Management.Automation.InvocationInfo MyCommand kubectl Path /opt/homebrew/bin/kubectl Extension Definition /opt/homebrew/bin/kubectl Source /opt/homebrew/bin/kubectl Version 0.0.0.0 Major 0 Minor 0 Build 0 Revision 0 MajorRevision 0 MinorRevision 0 Visibility Public OutputType index value 0 System.String Name System.String Type System.String TypeDefinitionAst <null> Name kubectl CommandType Application ModuleName Module <null> RemotingCapability PowerShell Parameters System.Management.Automation.PSNotSupportedException: Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column <filename unknown>:0:0\n" is not supported.\n at System.Management.Automation.CommandMetadata..ctor(CommandInfo commandInfo, Boolean shouldGen... ErrorRecord Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column <filename unknown>:0:0\n" is not supported. Exception System.Management.Automation.ParentContainsErrorRecordException: Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column <filename unknown>:0:0\n" is not supported. Message Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column :0:0
" is not supported. TargetSite <null> Data [ ] InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotImplemented: (:) [], ParentContainsErrorRecordException Category NotImplemented Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId NotSupported ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] TargetSite Void .ctor(System.Management.Automation.CommandInfo, Boolean) Name .ctor MemberType Constructor DeclaringType System.Management.Automation.CommandMetadata ReflectedType System.Management.Automation.CommandMetadata MetadataToken 100667362 Module System.Management.Automation.dll MDStreamVersion 131072 FullyQualifiedName /Users/david/.nuget/packages/microsoft.dotnet-interactive/1.0.632301/tools/net9.0/any/runtimes/unix/lib/net9.0/System.Management.Automation.dll ModuleVersionId 6124f3ae-295f-4d47-a897-fbada8c88

command terminated with exit code 1 Exception System.Management.Automation.RemoteException: command terminated with exit code 1 SerializedRemoteException <null> SerializedRemoteInvocationInfo <null> ErrorRecord command terminated with exit code 1 Exception System.Management.Automation.ParentContainsErrorRecordException: command terminated with exit code 1 Message command terminated with exit code 1 TargetSite <null> Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotSpecified: (:) [], ParentContainsErrorRecordException Category NotSpecified Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId RuntimeException ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] WasThrownFromThrowStatement False TargetSite <null> Message command terminated with exit code 1 Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotSpecified: (:) [], RemoteException Category NotSpecified Activity Reason RemoteException TargetName TargetType FullyQualifiedErrorId NativeCommandErrorMessage ErrorDetails <null> InvocationInfo System.Management.Automation.InvocationInfo MyCommand kubectl Path /opt/homebrew/bin/kubectl Extension Definition /opt/homebrew/bin/kubectl Source /opt/homebrew/bin/kubectl Version 0.0.0.0 Major 0 Minor 0 Build 0 Revision 0 MajorRevision 0 MinorRevision 0 Visibility Public OutputType index value 0 System.String Name System.String Type System.String TypeDefinitionAst <null> Name kubectl CommandType Application ModuleName Module <null> RemotingCapability PowerShell Parameters System.Management.Automation.PSNotSupportedException: Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column <filename unknown>:0:0\n" is not supported.\n at System.Management.Automation.CommandMetadata..ctor(CommandInfo commandInfo, Boolean shouldGen... ErrorRecord Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column <filename unknown>:0:0\n" is not supported. Exception System.Management.Automation.ParentContainsErrorRecordException: Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column <filename unknown>:0:0\n" is not supported. Message Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column :0:0
" is not supported. TargetSite <null> Data [ ] InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotImplemented: (:) [], ParentContainsErrorRecordException Category NotImplemented Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId NotSupported ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] TargetSite Void .ctor(System.Management.Automation.CommandInfo, Boolean) Name .ctor MemberType Constructor DeclaringType System.Management.Automation.CommandMetadata ReflectedType System.Management.Automation.CommandMetadata MetadataToken 100667362 Module System.Management.Automation.dll MDStreamVersion 131072 FullyQualifiedName /Users/david/.nuget/packages/microsoft.dotnet-interactive/1.0.632301/tools/net9.0/any/runtimes/unix/lib/net9.0/System.Management.Automation.dll ModuleVersionId 6124f3ae-295f-4d47-a897-fbada8c885b7 MetadataToken 1 ScopeName System.Management.Automation.dll Name System.Management.Automation.dll Assembly System.Management.Automation, Version=7.5.0.500, Culture=neutral, PublicKeyToken=31bf3856ad364e35 ModuleHandle System.ModuleHandle CustomAttributes [ [System.Security.UnverifiableCodeAttribute()], [System.Runtime.CompilerServices.RefSafetyRulesAttribute((Int32)11)] ] MethodHandle System.RuntimeMethodHandle Value 4536883264 Attributes Public, HideBySig, SpecialName, RT

Unable to use a TTY - input is not a terminal or the right kind of file Exception System.Management.Automation.RemoteException: Unable to use a TTY - input is not a terminal or the right kind of file SerializedRemoteException <null> SerializedRemoteInvocationInfo <null> ErrorRecord Unable to use a TTY - input is not a terminal or the right kind of file Exception System.Management.Automation.ParentContainsErrorRecordException: Unable to use a TTY - input is not a terminal or the right kind of file Message Unable to use a TTY - input is not a terminal or the right kind of file TargetSite <null> Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotSpecified: (:) [], ParentContainsErrorRecordException Category NotSpecified Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId RuntimeException ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] WasThrownFromThrowStatement False TargetSite <null> Message Unable to use a TTY - input is not a terminal or the right kind of file Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject Unable to use a TTY - input is not a terminal or the right kind of file CategoryInfo NotSpecified: (Unable to use a TTY… right kind of file:String) [], RemoteException Category NotSpecified Activity Reason RemoteException TargetName Unable to use a TTY - input is not a terminal or the right kind of file TargetType String FullyQualifiedErrorId NativeCommandError ErrorDetails <null> InvocationInfo System.Management.Automation.InvocationInfo MyCommand kubectl Path /opt/homebrew/bin/kubectl Extension Definition /opt/homebrew/bin/kubectl Source /opt/homebrew/bin/kubectl Version 0.0.0.0 Major 0 Minor 0 Build 0 Revision 0 MajorRevision 0 MinorRevision 0 Visibility Public OutputType index value 0 System.String Name System.String Type System.String TypeDefinitionAst <null> Name kubectl CommandType Application ModuleName Module <null> RemotingCapability PowerShell Parameters System.Management.Automation.PSNotSupportedException: Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column <filename unknown>:0:0\n" is not supported.\n at System.Management.Automation.CommandMetadata..ctor(CommandInfo commandInfo, Boolean shouldGen... ErrorRecord Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column <filename unknown>:0:0\n" is not supported. Exception System.Management.Automation.ParentContainsErrorRecordException: Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column <filename unknown>:0:0\n" is not supported. Message Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column :0:0
" is not supported. TargetSite <null> Data [ ] InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotImplemented: (:) [], ParentContainsErrorRecordException Category NotImplemented Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId NotSupported ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] TargetSite Void .ctor(System.Management.Automation.CommandInfo, Boolean) Name .ctor MemberType Constructor DeclaringType System.Management.Automation.CommandMetadata ReflectedType System.Management.Automation.CommandMetadata MetadataToken 100667362 Module System.Management.Automation.dll MDStreamVersion 131072 FullyQualifiedName /Users/david/.nuget/packages/microsoft.dotnet-interactive/1.0.632301/tools/net9.0/any/runtimes/unix/lib/net9.0/System.Management.Automation.dll ModuleVersionId 6124f3ae-295f-4d47-a897-fbada8c885b7 MetadataToken 1 ScopeName System.Management.Automation.dll Name System.Managem

rm: cannot remove '/config/Plug-in Support/Databases/com.plexapp.plugins.library.db-wal': No such file or directory Exception System.Management.Automation.RemoteException: rm: cannot remove '/config/Plug-in Support/Databases/com.plexapp.plugins.library.db-wal': No such file or directory SerializedRemoteException <null> SerializedRemoteInvocationInfo <null> ErrorRecord rm: cannot remove '/config/Plug-in Support/Databases/com.plexapp.plugins.library.db-wal': No such file or directory Exception System.Management.Automation.ParentContainsErrorRecordException: rm: cannot remove '/config/Plug-in Support/Databases/com.plexapp.plugins.library.db-wal': No such file or directory Message rm: cannot remove '/config/Plug-in Support/Databases/com.plexapp.plugins.library.db-wal': No such file or directory TargetSite <null> Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotSpecified: (:) [], ParentContainsErrorRecordException Category NotSpecified Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId RuntimeException ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] WasThrownFromThrowStatement False TargetSite <null> Message rm: cannot remove '/config/Plug-in Support/Databases/com.plexapp.plugins.library.db-wal': No such file or directory Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotSpecified: (:) [], RemoteException Category NotSpecified Activity Reason RemoteException TargetName TargetType FullyQualifiedErrorId NativeCommandErrorMessage ErrorDetails <null> InvocationInfo System.Management.Automation.InvocationInfo MyCommand kubectl Path /opt/homebrew/bin/kubectl Extension Definition /opt/homebrew/bin/kubectl Source /opt/homebrew/bin/kubectl Version 0.0.0.0 Major 0 Minor 0 Build 0 Revision 0 MajorRevision 0 MinorRevision 0 Visibility Public OutputType index value 0 System.String Name System.String Type System.String TypeDefinitionAst <null> Name kubectl CommandType Application ModuleName Module <null> RemotingCapability PowerShell Parameters System.Management.Automation.PSNotSupportedException: Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column <filename unknown>:0:0\n" is not supported.\n at System.Management.Automation.CommandMetadata..ctor(CommandInfo commandInfo, Boolean shouldGen... ErrorRecord Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column <filename unknown>:0:0\n" is not supported. Exception System.Management.Automation.ParentContainsErrorRecordException: Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column <filename unknown>:0:0\n" is not supported. Message Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column :0:0
" is not supported. TargetSite <null> Data [ ] InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotImplemented: (:) [], ParentContainsErrorRecordException Category NotImplemented Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId NotSupported ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] TargetSite Void .ctor(System.Management.Automation.CommandInfo, Boolean) Name .ctor MemberType Constructor DeclaringType System.Management.Automation.CommandMetadata ReflectedType System.Management.Automation.CommandMetadata MetadataToken 100667362 Module System.Management.Automation.dll MDStreamVersion 131072 FullyQualifiedName /Users/david/.nuget/packages/microsoft.dotnet-interactive/1.0.632301/tools/net9.0/any/runtimes/unix/lib/net9.0/System.Management.Automation.dll ModuleVersionId 6124f3ae-295f-4d47-a897-fbada8c88

command terminated with exit code 1 Exception System.Management.Automation.RemoteException: command terminated with exit code 1 SerializedRemoteException <null> SerializedRemoteInvocationInfo <null> ErrorRecord command terminated with exit code 1 Exception System.Management.Automation.ParentContainsErrorRecordException: command terminated with exit code 1 Message command terminated with exit code 1 TargetSite <null> Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotSpecified: (:) [], ParentContainsErrorRecordException Category NotSpecified Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId RuntimeException ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] WasThrownFromThrowStatement False TargetSite <null> Message command terminated with exit code 1 Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotSpecified: (:) [], RemoteException Category NotSpecified Activity Reason RemoteException TargetName TargetType FullyQualifiedErrorId NativeCommandErrorMessage ErrorDetails <null> InvocationInfo System.Management.Automation.InvocationInfo MyCommand kubectl Path /opt/homebrew/bin/kubectl Extension Definition /opt/homebrew/bin/kubectl Source /opt/homebrew/bin/kubectl Version 0.0.0.0 Major 0 Minor 0 Build 0 Revision 0 MajorRevision 0 MinorRevision 0 Visibility Public OutputType index value 0 System.String Name System.String Type System.String TypeDefinitionAst <null> Name kubectl CommandType Application ModuleName Module <null> RemotingCapability PowerShell Parameters System.Management.Automation.PSNotSupportedException: Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column <filename unknown>:0:0\n" is not supported.\n at System.Management.Automation.CommandMetadata..ctor(CommandInfo commandInfo, Boolean shouldGen... ErrorRecord Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column <filename unknown>:0:0\n" is not supported. Exception System.Management.Automation.ParentContainsErrorRecordException: Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column <filename unknown>:0:0\n" is not supported. Message Cannot perform operation because operation "NewNotSupportedException at offset 96 in file:line:column :0:0
" is not supported. TargetSite <null> Data [ ] InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotImplemented: (:) [], ParentContainsErrorRecordException Category NotImplemented Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId NotSupported ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] TargetSite Void .ctor(System.Management.Automation.CommandInfo, Boolean) Name .ctor MemberType Constructor DeclaringType System.Management.Automation.CommandMetadata ReflectedType System.Management.Automation.CommandMetadata MetadataToken 100667362 Module System.Management.Automation.dll MDStreamVersion 131072 FullyQualifiedName /Users/david/.nuget/packages/microsoft.dotnet-interactive/1.0.632301/tools/net9.0/any/runtimes/unix/lib/net9.0/System.Management.Automation.dll ModuleVersionId 6124f3ae-295f-4d47-a897-fbada8c885b7 MetadataToken 1 ScopeName System.Management.Automation.dll Name System.Management.Automation.dll Assembly System.Management.Automation, Version=7.5.0.500, Culture=neutral, PublicKeyToken=31bf3856ad364e35 ModuleHandle System.ModuleHandle CustomAttributes [ [System.Security.UnverifiableCodeAttribute()], [System.Runtime.CompilerServices.RefSafetyRulesAttribute((Int32)11)] ] MethodHandle System.RuntimeMethodHandle Value 4536883264 Attributes Public, HideBySig, SpecialName, RT

In [25]:
#!pwsh

Write-Host "🔄 Scaling Plex back up..." -ForegroundColor Cyan
kubectl scale deployment plex -n $namespace --replicas=1

Write-Host "`n⏳ Waiting for Plex to be ready..."
Start-Sleep -Seconds 5
kubectl wait --for=condition=Ready pod -n $namespace -l app.kubernetes.io/name=plex --timeout=300s

Write-Host "`n✅ Plex is back up!" -ForegroundColor Green
kubectl get pods -n $namespace -l app.kubernetes.io/name=plex

🔄 Scaling Plex back up...

⏳ Waiting for Plex to be ready...

✅ Plex is back up!
deployment.apps/plex scaled
pod/plex-ccf674f6c-csm2n condition met
NAME                   READY   STATUS    RESTARTS   AGE
plex-ccf674f6c-csm2n   1/1     Running   0          70s


In [27]:
#!pwsh

Write-Host "🧹 Cleaning up debug pod..." -ForegroundColor Yellow
kubectl delete pod $podName -n $namespace --wait=true

Write-Host "✅ Debug pod deleted" -ForegroundColor Green

🧹 Cleaning up debug pod...
✅ Debug pod deleted


Error from server (NotFound): pods "plex-debug" not found Exception System.Management.Automation.RemoteException: Error from server (NotFound): pods "plex-debug" not found SerializedRemoteException <null> SerializedRemoteInvocationInfo <null> ErrorRecord Error from server (NotFound): pods "plex-debug" not found Exception System.Management.Automation.ParentContainsErrorRecordException: Error from server (NotFound): pods "plex-debug" not found Message Error from server (NotFound): pods "plex-debug" not found TargetSite <null> Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotSpecified: (:) [], ParentContainsErrorRecordException Category NotSpecified Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId RuntimeException ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] WasThrownFromThrowStatement False TargetSite <null> Message Error from server (NotFound): pods "plex-debug" not found Data (empty) InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject Error from server (NotFound): pods "plex-debug" not found CategoryInfo NotSpecified: (Error from server (…ex-debug" not found:String) [], RemoteException Category NotSpecified Activity Reason RemoteException TargetName Error from server (NotFound): pods "plex-debug" not found TargetType String FullyQualifiedErrorId NativeCommandError ErrorDetails <null> InvocationInfo System.Management.Automation.InvocationInfo MyCommand kubectl Path /opt/homebrew/bin/kubectl Extension Definition /opt/homebrew/bin/kubectl Source /opt/homebrew/bin/kubectl Version 0.0.0.0 Major 0 Minor 0 Build 0 Revision 0 MajorRevision 0 MinorRevision 0 Visibility Public OutputType index value 0 System.String Name System.String Type System.String TypeDefinitionAst <null> Name kubectl CommandType Application ModuleName Module <null> RemotingCapability PowerShell Parameters System.Management.Automation.PSNotSupportedException: Cannot perform operation because operation "NewNotSupportedException at offset 88 in file:line:column <filename unknown>:0:0\n" is not supported.\n at System.Management.Automation.CommandMetadata..ctor(CommandInfo commandInfo, Boolean shouldGen... ErrorRecord Cannot perform operation because operation "NewNotSupportedException at offset 88 in file:line:column <filename unknown>:0:0\n" is not supported. Exception System.Management.Automation.ParentContainsErrorRecordException: Cannot perform operation because operation "NewNotSupportedException at offset 88 in file:line:column <filename unknown>:0:0\n" is not supported. Message Cannot perform operation because operation "NewNotSupportedException at offset 88 in file:line:column :0:0
" is not supported. TargetSite <null> Data [ ] InnerException <null> HelpLink <null> Source <null> HResult -2146233087 StackTrace <null> TargetObject <null> CategoryInfo NotImplemented: (:) [], ParentContainsErrorRecordException Category NotImplemented Activity Reason ParentContainsErrorRecordException TargetName TargetType FullyQualifiedErrorId NotSupported ErrorDetails <null> InvocationInfo <null> ScriptStackTrace <null> PipelineIterationInfo [ ] TargetSite Void .ctor(System.Management.Automation.CommandInfo, Boolean) Name .ctor MemberType Constructor DeclaringType System.Management.Automation.CommandMetadata ReflectedType System.Management.Automation.CommandMetadata MetadataToken 100667362 Module System.Management.Automation.dll MDStreamVersion 131072 FullyQualifiedName /Users/david/.nuget/packages/microsoft.dotnet-interactive/1.0.632301/tools/net9.0/any/runtimes/unix/lib/net9.0/System.Management.Automation.dll ModuleVersionId 6124f3ae-295f-4d47-a897-fbada8c885b7 MetadataToken 1 ScopeName System.Management.Automation.dll Name System.Management.Automation.dll Assembly System.Management.Automation, Version=7.5.0.500, Culture=neutral, PublicKeyToken=31b